In [ ]:
import pandas as pd
import numpy as np 
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from kaggle_datasets import KaggleDatasets

PATH = "../input/celeba-dataset/img_align_celeba/img_align_celeba"

In [ ]:
df_path = "../input/celeba-dataset/list_attr_celeba.csv"
celeb_df = pd.read_csv(df_path)
celeb_df.head()

In [ ]:
unattr_female = celeb_df[(celeb_df["Attractive"] == -1) & (celeb_df["Male"] == -1)]
unattr_female = unattr_female["image_id"]
print(len(unattr_female))

In [ ]:
@tf.function
def preprocessing(img_path):
    img_path = PATH + "/" + img_path
    img = tf.io.read_file(img_path) 
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [64, 64])

    img = tf.cast(img, tf.float32) / 255.0

    return img

@tf.function
def create_dataset(df, batch_size):
    ds = tf.data.Dataset.from_tensor_slices((df))
    ds = ds.map(preprocessing, tf.data.AUTOTUNE)

    train_dataset = ds.cache().batch(BATCH_SIZE)
    train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    return train_dataset

In [ ]:
# TRAIN_LENGTH = len(unattr_female)
BATCH_SIZE = 64
# BUFFER_SIZE = 3500
# STEPS_PER_EPOCH = TRAIN_LENGTH // BATCH_SIZE

In [ ]:
train_dataset = create_dataset(unattr_female, BATCH_SIZE)

In [ ]:
for data in train_dataset.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(data[i].numpy())
        plt.axis("off")

In [ ]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(8*8*256, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((8, 8, 256)))
    assert model.output_shape == (None, 8, 8, 256)  # Note: None is the batch size

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 16, 16, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 32, 32, 64)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(32, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 64, 64, 32)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(3, (5, 5), strides=(1, 1), padding='same', use_bias=False, activation="tanh"))
    assert model.output_shape == (None, 64, 64, 3)


    return model

In [ ]:
generator = make_generator_model()
generator.summary()

In [ ]:
noise = tf.random.normal([1, 100])
generated_image = generator(noise, training=False)

plt.imshow(generated_image[0]*255)

In [ ]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same',
                                     input_shape=[64, 64, 3]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))


    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model

In [ ]:
discriminator = make_discriminator_model()
discriminator.summary()

In [ ]:
decision = discriminator(generated_image)
print(decision)

In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [ ]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [ ]:
EPOCHS = 500
noise_dim = 100
num_examples_to_generate = 9

seed = tf.random.normal([num_examples_to_generate, noise_dim])

In [ ]:
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [ ]:
def display_images():
    predictions = generator(seed, training=False)
    plt.figure()
    for i in range(predictions.shape[0]):
        plt.subplot(3, 3, i+1)
        plt.imshow((predictions[i]* 255).numpy().astype(np.uint8))
        plt.axis('off')
    plt.show()

In [ ]:
import time
def train(dataset, epochs):
    for epoch in range(epochs):
        start = time.time()

        for image_batch in dataset:
            train_step(image_batch)

        print('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))
        
        if (epoch+1) % 20 == 0:
            display_images()

In [ ]:
train(train_dataset, EPOCHS)